In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [27]:
def get_order_parameter(theta,N):
    ''' get theta and return r and theta'''
    rpsi = 1/N*np.sum(np.exp(1j*theta))
    r = np.abs(rpsi)
    psi = np.angle(rpsi)
    return r,psi

In [ ]:
def RK4(f, y0, t, args=()):


In [47]:
    
def Sol_Kuramoto_r(K,N,m,t_array,p_theta = [], p_dtheta = [], p_omega = []):
    """
    멀티프로세스를 효율적으로 돌리기 위한 시스템 경량화
    """
    theta, dtheta, omega  =  p_theta, p_dtheta,p_omega
    n = len(t_array)
    rs = np.zeros((n))
    r,psi = get_order_parameter(theta,N)
    rs[0] = r
    h = t_array[i + 1] - t_array[i]
    args=(omega,N,m,K)
    y_temp = np.r_[theta,dtheta,r,psi]
    if m==0:
        f = Kuramoto_1st_mf
    else:
        f = Kuramoto_2nd_mf
    for i in range(n - 1):
        
        t_temp = t_array[i]
        k1 = f(y_temp, t_temp, *args)
        k2 = f(y_temp + k1 * h / 2.0, t_temp + h / 2.0, *args)
        k3 = f(y_temp + k2 * h / 2.0, t_temp + h / 2.0, *args)
        k4 = f(y_temp + k3 * h, t_temp + h, *args)
        y_temp= y_temp + (h / 6.0) * (k1 + 2 * k2 + 2 * k3 + k4)
        rs[i] = y_temp[-2]

    return rs
def Kuramoto_2nd_mf(Theta,t,omega,N,m,K):
    # print(f"Case m = {m}") 
    theta,dtheta,r,psi = Theta[:N],Theta[N:2*N],Theta[-2],Theta[-1]
    ddtheta = (-dtheta + omega + K*r*np.sin(psi - theta))
    r,psi = get_order_parameter(dtheta,N)
    return np.r_[dtheta,ddtheta,r,psi]

def Kuramoto_1st_mf(Theta,t,omega,N,m,K):
    # print("Case m = 0")
    theta,r,psi = Theta[:N],Theta[-2],Theta[-1]
    dtheta = omega + K*r*np.sin(psi - theta)
    r,psi = get_order_parameter(dtheta,N)
    return np.r_[dtheta,np.zeros(N),r,psi]

def Sol_Kuramoto_sampling(N,K,m,t_array,t_sample_idx,p_theta = [], p_dtheta = [], p_omega = []):
    """
    멀티프로세스를 효율적으로 돌리기 위한 시스템 경량화
    """
    theta, dtheta, omega  =  p_theta, p_dtheta,p_omega
            
    result = RK4_sampling(Kuramoto_2nd_mf,np.r_[theta,dtheta],t_array,t_sample_idx,args=(omega,N,m,K))
    theta_s = result[:,:N]
    dtheta_s = result[:,N:]
    return theta_s,dtheta_s

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from TO_sim.Sol_Kuramoto import Sol_Kuramoto_theta_dtheta as sim
from TO_sim.Sol_Kuramoto import Sol_Kuramoto_sampling as sampling
from TO_sim.Sol_Kuramoto import Make_order_parameter as make_r
from TO_sim.Utility import Create_Folder
from TO_sim.Kuramoto_model import Kuramoto_2nd_mf as func
from TO_sim.gen_Distribution import Normal
import scipy.integrate as SI
import parmap
import pandas as pd
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable


In [42]:
N =5000
theta,omega,Kc = Normal(N,0,1,seed=10)
omega = np.sort(omega)
theta0 = np.ones_like(theta_random)

In [43]:
%timeit np.array([*theta,*omega])

378 µs ± 3.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [44]:
%timeit np.r_[theta,omega]

7.2 µs ± 26.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [46]:
np.r_[theta,omega,0,1].shape

(10002,)

In [35]:
A = np.r_[theta,omega]

In [24]:
%timeit a,b = A[:N],A[N:2*N]

146 ns ± 0.933 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [36]:
theta,dtheta = A[:N],A[N:2*N]
r,psi = get_order_parameter(theta,N)
ddtheta = (-dtheta + omega + K*r*np.sin(psi - theta))
A = np.r_[dtheta,ddtheta,r,psi]

In [37]:
A[-2]

0.01999916553513436

In [38]:
%timeit r,psi = get_order_parameter(theta,N)

9.95 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [39]:
%timeit V = np.var(dtheta)

4.97 µs ± 23.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [40]:
%timeit np.array([*theta,*dtheta])

38.9 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [41]:
%timeit np.r_[theta,dtheta]


5.79 µs ± 24.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
Sol_Kuramoto_r()